In [44]:
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient
from azure.ai.ml import command, Input
from azure.ai.ml.sweep import Choice, Uniform, MedianStoppingPolicy
from azure.identity import DefaultAzureCredential

In [4]:
credential = DefaultAzureCredential()
ml_client = None
try:
    ml_client = MLClient.from_config(credential)
except Exception as ex:
    print(ex)
    # Enter details of your Azure Machine Learning workspace
    subscription_id = 'd86ba508-6a09-4959-8b9f-f076ad827658'
    resource_group = 'cloud-computing-module-iv'
    workspace_name = 'cloud-computing-project'
    ml_client = MLClient(credential, subscription_id, resource_group, workspace_name)

NameError: name 'DefaultAzureCredential' is not defined

In [ ]:
from azure.ai.ml import command, Input

# define the command
command_job = command(
    code="./src",
    command="python main.py --titanic-csv ${{inputs.titanic_csv}} --learning-rate ${{inputs.learning_rate}} --boosting ${{inputs.boosting}} --num-leaves ${{inputs.num_leaves}}",
    environment="AzureML-lightgbm-3.2-ubuntu18.04-py37-cpu@latest",
    inputs={
        "titanic_csv": Input(
            type="uri_file",
            path="./data/processed_titanic_data.csv",
        ),
        "learning_rate": 0.9,
        "boosting": "gbdt",
        "num_leaves": 31,
    },
    compute="cpu-cluster",
)



In [58]:
# submit the command
returned_job = ml_client.jobs.create_or_update(command_job)
# get a URL for the status of the job
returned_job.studio_url

Uploading proyecto-I (0.91 MBs): 100%|██████████| 908367/908367 [00:00<00:00, 3714059.19it/s]




'https://ml.azure.com/runs/bubbly_cushion_33p2bz20mt?wsid=/subscriptions/d86ba508-6a09-4959-8b9f-f076ad827658/resourcegroups/cloud-computing-module-iv/workspaces/cloud-computing-project&tid=3cc9ebfe-6684-4004-9d2d-9988b42cbf67'

In [61]:
command_job_for_sweep = command_job(
    learning_rate=Uniform(min_value=0.01, max_value=0.9),
    boosting=Choice(values=["gbdt", "dart"]),
    num_leaves=Choice(values=[31, 35, 39, 43]),
)

In [62]:
# apply the sweep parameter to obtain the sweep_job
sweep_job = command_job_for_sweep.sweep(
    compute="cpu-cluster",
    sampling_algorithm="random",
    primary_metric="binary_error",
    goal="Maximize",
)

# define the limits for this sweep
sweep_job.set_limits(max_total_trials=20, max_concurrent_trials=10, timeout=7200)

In [63]:
# submit the sweep
returned_sweep_job = ml_client.create_or_update(sweep_job)
# get a URL for the status of the job
returned_sweep_job.studio_url

'https://ml.azure.com/runs/silly_airport_jvwvn72j09?wsid=/subscriptions/d86ba508-6a09-4959-8b9f-f076ad827658/resourcegroups/cloud-computing-module-iv/workspaces/cloud-computing-project&tid=3cc9ebfe-6684-4004-9d2d-9988b42cbf67'